In [2]:
from firebase import firebase
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from firebase import firebase
import time
import pandas as pd
import numpy as np

In [ ]:
def newRecord(record):
    newRes = record[2]
    while record[0] == newRes:
        firebase_ = firebase.FirebaseApplication('https://hotelwishlist-86f4f.firebaseio.com/', None)
        result = firebase_.get('/info', None)

        time = {}
        for key in result.keys():
            #print(result[key], result[key]['primaryKey'])
            time[key] = result[key]['primaryKey']

        res = {k: v for k, v in sorted(time.items(), key=lambda item: item[1], reverse=True)}

        newRes = result[list(res.keys())[0]]
    return newRes

In [1]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    record = ['', firebase, '']
    while True:
        record[2] = record[0]
        future = executor.submit(newRecord, record)
        return_value = future.result()
        #print(return_value)
        record[0] = return_value
        if(record[0] != record[2]):
            # Write every MainCode file here ...
            #print(record[0])
            #print(record[0]['source'], record[0]['destination'], record[0]['inDate'], record[0]['primaryKey'])
            browser = webdriver.Chrome("C:\\Users\\Snehal\\PycharmProjects\\RL\\Solve-for-tomorrow\\chromedriver.exe")
            df = pd.read_csv(r'data/city_codes.csv')
            #location = 'Delhi'
            location = record[0]['location']
            location = location.lower().capitalize()
            locationCode = df['iata_code'][df[df['city'] == location].index.tolist()].tolist()[0]
            #checkin = '08/07/2020' #dd/mm/yyyy
            checkin = record[0]['inDate']
            tempDate = checkin.split('-')
            checkin_url = tempDate[1] + tempDate[2] + tempDate[0]
            #checkout = '09/07/2020' #dd/mm/yyyy
            checkout = record[0]['outDate']
            tempDate = checkout.split('-')
            checkout_url = tempDate[1] + tempDate[2] + tempDate[0]
            browser.implicitly_wait(20)
            url = 'https://www.makemytrip.com/hotels/hotel-listing/?checkin=' + checkin_url + '&checkout=' + checkout_url + '&locusId=CT' + locationCode + '&locusType=city&city=CT' + locationCode + '&country=IN&searchText=' + location + '&roomStayQualifier=2e0e&_uCurrency=INR&reference=hotel&type=city'
            browser.get(url)
            # Scroll the page till bottom to get full data available in the DOM.
            print ("Scrolling document upto bottom ...")
            for j in range(1, 10):
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
            body = browser.find_element_by_tag_name("body").get_attribute("innerHTML")
            print("Getting data from DOM ...")
            soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
            pHotelName = soupBody.findAll("p", {"class": "latoBlack font22 blackText appendBottom12"}) 
            hotelsNameData=[]
            for j in range(0, len(pHotelName)):
                hotelsNameData.append(pHotelName[j].text)
            pHotelPrice = soupBody.findAll("p", {"class": "latoBlack font26 blackText appendBottom5"}) 
            hotelsPriceData=[]
            for j in range(0, len(pHotelPrice)):
                hotelsPriceData.append(pHotelPrice[j].text)
            scraped_data = []
            for i in range(len(hotelsPriceData)):
                # initialize the dictionary
                card_details = {}
                # add data to the dictionary
                card_details['hotel_name'] = hotelsNameData[i]
                card_details['room_price'] = hotelsPriceData[i]
                # append the scraped data to the list
                scraped_data.append(card_details)
                # create a data frame from the list of dictionaries
            dataFrame = pd.DataFrame.from_dict(scraped_data)
            dataFrame.insert(0, 'location', location)
            dataFrame.insert(0, 'index', range(0, len(dataFrame)))
            print(dataFrame)
            json_records = dataFrame.to_json(r'result.json',orient ='records') 
            json_records = dataFrame.to_dict(orient ='records')
            print(json_records)
            firebase_ = firebase.FirebaseApplication('https://hotelwishlist-86f4f.firebaseio.com/', None)
            firebase_.delete('/hotel/', '')
            for record_ in json_records:
                print(record_)
                result = firebase_.post('/hotel/',record_)
            
            
            

NameError: name 'concurrent' is not defined